### FAP tests

We are testing against the null hypothesis that the white dwarf found was a chance alignment of a random source that just happened to meet our criteria (i.e. appears co-moving with the pulsar, has properties consistent with being a WD) with the pulsar position on the sky.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.coordinates import SkyCoord, Angle
from astropy.time import Time
from astropy.table import Table, unique
from tqdm import tqdm
from astroquery.gaia import Gaia
from scipy.special import erfinv, erfcinv, erf, erfc

In [ ]:
pulsar_data = np.genfromtxt('ipta_3arcsec_wpm-result.csv', 
                            delimiter=',', names=True, dtype=None, encoding=None)

In [ ]:
pulsar_data = Table(pulsar_data)
pulsar_data.sort(['names', 'angdist'])
#pulsar_data = unique(pulsar_data, keys='names') # drop less-good duplicate matches

This function will reproduce the cuts that we made to determine which nearby sources were candidate matches and which should be thrown out:

**NOTE: absolute magnitude > 7 cut was made relatively arbitrarily to weed out main-sequence stars from white dwarfs - this may need to be changed depending on what criteria we settle on for the paper.**

In [ ]:
def make_cuts(tbl, pulsar_pmra, pulsar_pmdec):
    # take an astropy table, apply cuts
    pmra_tol = 3. * tbl['pmra_error'] # 3 sigma
    pmdec_tol = 3. * tbl['pmdec_error'] # 3 sigma
    idx = np.isfinite(tbl['parallax']) & (np.abs(tbl['pmra'] - pulsar_pmra) <= pmra_tol) \
            & (np.abs(tbl['pmdec'] - pulsar_pmdec) <= pmdec_tol) 
#            & ((tbl['phot_g_mean_mag'] + 5.*np.log10(tbl['parallax']) - 10.) >= 5.)
    return tbl[idx]

In [ ]:
make_cuts(pulsar_data, 
          pulsar_data['pmra_1'], 
          pulsar_data['pmdec_1'])['names']

Now we'll randomize each pulsar's coordinates (within 3 degrees in RA and Dec to reflect the actual crowding in the general sky region) and see how many objects in the new area pass our cuts.

In [ ]:
pulsars_to_test = make_cuts(pulsar_data, 
          pulsar_data['pmra_1'], 
          pulsar_data['pmdec_1']) # just the good matches
faps = []
N_trials = 10000
search_radius = u.Quantity(3.0, u.arcsec)
p  = pulsars_to_test[3] # do one pulsar at a time
print('calculating FAP for pulsar {0}...'.format(p['names']))
n_found = np.zeros(N_trials)
for n in tqdm(range(N_trials)):
    test_ra = p['ra_1'] + np.random.normal(0., 3.) # randomized within 3 degrees
    test_dec = p['dec_1'] + np.random.normal(0., 3.) # randomized within 3 degrees
    test_coord = SkyCoord(ra=test_ra, dec=test_dec, unit=(u.degree, u.degree))
    j = Gaia.cone_search_async(test_coord, search_radius, verbose=False)
    r = j.get_results()
    n_found[n] = len(make_cuts(r, p['pmra_1'], p['pmdec_1']))
    #print('found FAP of {0} in {1}.'.format(np.sum(n_found > 0), N_trials))
    f=open('FAPs/FAP_J1732-5049.txt','ab')
    np.savetxt(f,n_found)
    f.close()
faps.append(np.sum(n_found > 0)/N_trials)
    
    


In [ ]:
for p,f in zip(pulsars_to_test, faps):
    print("{0}: FAP {1}".format(p['names'], f))

# FAP of J1843

In [ ]:
foundfile1843 = np.loadtxt("FAPs/FAP_J1843-1113.txt")

In [ ]:
FAP_1843 = np.sum(foundfile1843)/len(foundfile1843)

In [ ]:
len(foundfile1843)

In [ ]:
"%.2e"%FAP_1843

In [ ]:
print("This is a {} detection".format(erfcinv(FAP_1843)*np.sqrt(2)))

# FAP of J0437

In [ ]:
foundfile0437 = np.loadtxt("FAPs/FAP_J0437.txt")

In [ ]:
"%.2e"%(1/len(foundfile0437))

In [ ]:
FAP_0437 = np.sum(foundfile0437)/len(foundfile0437)
print(FAP_0437)

In [ ]:
print("This is greater than a {} detection".format(erfcinv(1/len(foundfile0437))*np.sqrt(2)))

# FAP of J1012

In [ ]:
foundfile1012 = np.loadtxt("FAPs/FAP_J1012.txt")

In [ ]:
"%.2e"%len(foundfile1012)

In [ ]:
FAP_1012 = np.sum(foundfile1012)/len(foundfile1012)
print(FAP_1012)

In [ ]:
print("This is greater than a {} detection".format(erfcinv(1/len(foundfile1012))*np.sqrt(2)))

# FAP of J1024

In [ ]:
foundfile1024 = np.loadtxt("FAPs/FAP_J1024.txt")

In [ ]:
FAP_1024 = np.sum(foundfile1024)/len(foundfile1024)
print("FAP of J1024 is less than {0}".format(1/len(foundfile1024)))

In [ ]:
print("This is greater than a {} detection".format(erfcinv(1/len(foundfile1024))*np.sqrt(2)))

# FAP of J1732

In [ ]:
foundfile1732 = np.loadtxt("FAPs/FAP_J1732-5049.txt")

In [ ]:
FAP_1732 = np.sum(foundfile1732)/len(foundfile1732)
print("%.2e"%FAP_1732)

In [ ]:
print("This is a {} detection".format(erfcinv(FAP_1732)*np.sqrt(2)))

# FAP of J1910

In [ ]:
foundfile1910 = np.loadtxt("FAPs/FAP_J1910+1256.txt")

In [ ]:
"%.2e"%len(foundfile1910)

In [ ]:
FAP_1910 = np.sum(foundfile1910)/len(foundfile1910)
print("%.2e"%FAP_1910)

In [ ]:
print("This is a {} detection".format(erfcinv(FAP_1910)*np.sqrt(2)))